# Movement Range Maps
This notebook describes the processing of movement data made available by the Humanitarian Data Exchange. The data includes movement changes measured by Facebook throughout March to August 2020 starting from a baseline in February.


## Data Description
The location data from is obtained from Facebook users with care taken to ensure individual users are not identifiable. From these data we can compute the following metrics.

1. Change in movement metric - Number of 600 meters by 600 meters tiles people are seen in within a day.
2. Stay put metric - Percentage of people who are only observed in a single 600 meters by 600 meters tile

## Data Download

We obtain the data from https://data.humdata.org/dataset/movement-range-maps

The data is contained in a zip file which contains a tab limited file with the data.

In [ ]:
import urllib.request
import zipfile
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets


from IPython.display import display


data_url = 'https://data.humdata.org/dataset/c3429f0e-651b-4788-bb2f-4adbf222c90e/resource/31ca909c-10d9-458a-8720-88b54b3e3627/download/movement-range-data-2020-08-25.zip'

# download
urllib.request.urlretrieve(data_url,
                           'movement-range-data-2020-08-25.zip')

# unzip
with zipfile.ZipFile('movement-range-data-2020-08-25.zip', 'r') as zip_ref:
    zip_ref.extractall('./')



We now load the data using pandas and examine it

In [ ]:
movement_data = pd.read_csv('movement-range-2020-08-25.txt', 
                             sep="\t", 
                             low_memory=False)

In [ ]:
# explore the data
movement_data.head()

In [ ]:
# number of entries
len(movement_data)

The movement metrics are stored under the following columns
1. Change of movement - 'all_day_bing_tiles_visited_relative_change'
2. Stay put - 'all_day_ratio_single_tile_users'

To obtain data from a given location, we filter the data using the `country` and `polygon_name` columns. 

In [ ]:
# choose a country
countries = set(movement_data['country'])
country_dd = widgets.Dropdown(
                options=countries,
                value='KEN',
                description='Country:',
                disabled=False,
            )
display(country_dd)

In [ ]:
# choose a location
country_data = movement_data[movement_data['country'] == country_dd.value]
locations = set(country_data['polygon_name'])

location_dd = widgets.Dropdown(
                options=locations,
                value=None,
                description='Location:',
                disabled=False,
            )
display(location_dd)

In [ ]:
# Get the data for the given location
curr_df = country_data[country_data['polygon_name'] == location_dd.value]
curr_df[['ds']] = curr_df[['ds']].apply(pd.to_datetime)

#change of movement
days = curr_df['ds']
com = curr_df['all_day_bing_tiles_visited_relative_change']

plt.figure()
plt.plot(days, com);
plt.grid(True)
plt.title(location_dd.value, fontdict={'fontsize': 18});
plt.ylabel('Change of Movement')

if country_dd.value == 'KEN':
    plt.vlines(datetime.datetime(2020,3,13), min(com), max(com) + .1,'m')
    plt.vlines(datetime.datetime(2020,4,6), min(com), max(com) + .1,'r')
    plt.vlines(datetime.datetime(2020,7,6), min(com), max(com) + .1,'g')

# stay put index
sp = curr_df['all_day_ratio_single_tile_users']
plt.figure()
plt.plot(days, sp);
plt.grid(True)
plt.title(location_dd.value, fontdict={'fontsize': 18});
plt.ylabel('Stay Put Index');

if country_dd.value == 'KEN':
    plt.vlines(datetime.datetime(2020,3,13), min(sp), max(sp) + .1,'m')
    plt.vlines(datetime.datetime(2020,4,6), min(sp), max(sp) + .1,'r')
    plt.vlines(datetime.datetime(2020,7,6), min(sp), max(sp) + .1,'g')